In [ ]:
# !pip install selenium

In [ ]:
# !pip install webdriver_manager

# 자동로그인을 하기 위한 사용자 정보 수집 코드 chrome_profile

In [9]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# --- Selenium 옵션 설정 ---
options = Options()
options.add_experimental_option("detach", True)

# ★★★ 사용자 프로필 경로 설정 ★★★
profile_path = os.path.join(os.getcwd(), "chrome_profile")
options.add_argument(f"--user-data-dir={profile_path}")

# --- 자동화 탐지 회피 설정 ---
options.add_argument("start-maximized")
# ... (이전과 동일한 나머지 옵션들) ...
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Whale/4.32.315.15 Safari/537.36")
options.add_argument("referrer-policy=unsafe-url")
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("lang=ko_KR")

# 드라이버 실행
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

# 네이버 메인으로 이동
driver.get("https://www.naver.com")

# ★★★★★ 가장 중요한 부분 ★★★★★
input("""
브라우저가 열렸습니다. 다음 단계를 꼭 순서대로 진행해주세요:
1. ID와 비밀번호를 입력합니다.
2. [로그인 상태 유지] 체크박스를 ★반드시★ 선택합니다.
3. [로그인] 버튼을 클릭합니다.
4. 로그인이 완료된 것을 확인한 후, 이 터미널로 돌아와 Enter 키를 누르세요...
""")

print("로그인 프로필이 저장되었습니다. 브라우저를 닫으셔도 됩니다.")
print("이제부터는 '세션 재사용' 코드를 실행해주세요.")

driver.quit()



브라우저가 열렸습니다. 다음 단계를 꼭 순서대로 진행해주세요:
1. ID와 비밀번호를 입력합니다.
2. [로그인 상태 유지] 체크박스를 ★반드시★ 선택합니다.
3. [로그인] 버튼을 클릭합니다.
4. 로그인이 완료된 것을 확인한 후, 이 터미널로 돌아와 Enter 키를 누르세요...

로그인 프로필이 저장되었습니다. 브라우저를 닫으셔도 됩니다.
이제부터는 '세션 재사용' 코드를 실행해주세요.


In [11]:
import os
import time
import csv
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException


def setup_driver():
    """Chrome WebDriver를 설정하고 반환합니다."""
    options = Options()
    profile_path = os.path.join(os.getcwd(), "chrome_profile")
    options.add_argument(f"--user-data-dir={profile_path}")
    options.add_experimental_option("detach", True)
    options.add_argument("start-maximized")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    # 최신 Chrome User-Agent로 업데이트 (크롤링 시점의 최신 안정 버전으로 조정 권장)
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36")
    options.add_argument('--disable-blink-features=AutomationControlled') # 봇 감지 우회
    options.add_argument("lang=ko_KR") # 언어 설정

    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        driver.implicitly_wait(5) # 요소를 찾을 때까지 최대 5초 대기
        return driver
    except Exception as e:
        print(f"🚨 드라이버 설정 중 오류 발생: {e}")
        exit() # 드라이버 설정 실패 시 프로그램 종료


def parse_cafe_date(date_str, collected_date):
    """카페 게시물 날짜 문자열을 파싱합니다. 'HH:MM' 형식은 오늘 날짜로 처리합니다."""
    try:
        if ":" in date_str:
            return collected_date # 시간만 있으면 오늘 날짜로 간주
        else:
            return datetime.strptime(date_str.strip(), '%Y.%m.%d.').date()
    except ValueError:
        print(f"⚠️ 날짜 형식 파싱 오류: '{date_str}'")
        return collected_date # 파싱 실패 시 수집된 날짜 반환 (오늘 날짜)


def safe_wait(driver, condition, timeout=20, retries=3, delay=2):
    """주어진 조건이 만족될 때까지 안전하게 기다립니다. 타임아웃 시 재시도합니다."""
    for attempt in range(retries):
        try:
            wait = WebDriverWait(driver, timeout)
            element = wait.until(condition)
            # print(f"✅ 요소 로드 성공 (시도 {attempt+1}/{retries})", end="\r")
            return element
        except TimeoutException:
            print(f"⏳ 요소 로딩 지연 (시도 {attempt+1}/{retries}) - 조건: {condition}", end="\r")
            time.sleep(delay)
        except Exception as e:
            print(f"⚠️ safe_wait 중 예외 발생: {e}", end="\r")
            time.sleep(delay)
    raise TimeoutException(f"⚠️ 요소 로딩 최종 실패: {condition}")


def handle_login(driver, wait):
    """로그인 버튼이 보이면 로그인 시도를 합니다."""
    try:
        # 'gnb_login_button'이 로그인 상태가 아닐 때만 나타남
        login_btn = safe_wait(driver, EC.element_to_be_clickable((By.ID, "gnb_login_button")), timeout=5, retries=1)
        if login_btn: # 요소가 성공적으로 찾아지면 클릭
            print("🔐 로그인 버튼 발견 → 로그인 시도 (ID/PW 입력 필요)", end="\r")
            login_btn.click()
            time.sleep(1) # 로그인 페이지 로드 대기
            # 'pw' 필드는 실제로 입력해야 하므로, 여기서는 그냥 ENTER만 칩니다.
            # 실제 로그인하려면 ID/PW를 입력하는 코드가 필요합니다.
            try:
                pw_box = wait.until(EC.presence_of_element_located((By.ID, "pw")))
                pw_box.send_keys(Keys.ENTER) # ID/PW를 이미 저장했다면 ENTER로 넘어갈 수 있음
                print("🔐 비밀번호 입력창 발견 → ENTER 시도", end="\r")
                time.sleep(5) # 로그인 처리 대기
            except TimeoutException:
                print("⚠️ 비밀번호 입력창을 찾을 수 없습니다.", end="\r")
            except Exception as e:
                print(f"⚠️ 로그인 필드 상호작용 중 오류 발생: {e}", end="\r")
        else:
            print("✅ 로그인 버튼 비활성화 상태 → 로그인 유지 중", end="\r")
    except TimeoutException: # 로그인 버튼이 일정 시간 내에 나타나지 않으면 로그인 상태로 간주
        print("✅ 로그인 버튼 없음 (로그인 상태로 추정)", end="\r")
    except Exception as e:
        print(f"⚠️ 로그인 처리 중 예외 발생: {e}", end="\r")


# --- 메인 크롤링 로직 시작 ---
print("네이버카페 게시판 크롤링 시작")
driver = setup_driver()
wait = WebDriverWait(driver, 20) # 명시적 대기 시간

base_url = "https://cafe.naver.com/f-e/cafes/19997929/menus/809"
csv_filename = "test" #'보만사_results_1year_inline.csv'

with open(csv_filename, 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    writer.writerow(['작성일', '제목', '본문', '댓글', 'URL'])

    one_year_ago = (datetime.now() - timedelta(days=3)).date() # 2년 전으로 설정
    page_count = 1
    stop_collecting = False

    while not stop_collecting:
        target_url = f"{base_url}?page={page_count}&size=50" # 페이지 번호와 사이즈 파라미터 추가

        # google.com 튕김 방지 및 초기 진입
        current_url = driver.current_url
        if "google.com" in current_url or "naver.com" not in current_url:
            print("🚨 비정상 URL 감지 또는 초기 진입 → 대상 URL로 이동")
            driver.get(target_url)
            time.sleep(3) # 페이지 로드 대기
            handle_login(driver, wait) # 로그인 처리 시도

        print(f"\n📄 페이지 {page_count} 진입 시도: {target_url}")
        # 매 페이지 이동 전에 명시적으로 URL을 설정하여 확실하게 이동
        driver.get(target_url)
        time.sleep(3) # 페이지 로드 대기
        handle_login(driver, wait) # 로그인 처리 시도

        try:
            # 게시물 목록의 tbody를 찾습니다.
            # .article-table 클래스를 가진 테이블 내의 tbody를 찾습니다.
            # 여러 tbody가 있을 수 있으므로 마지막 tbody를 선택
            tbodys = safe_wait(driver, EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table.article-table tbody")), timeout=30)
            if not tbodys:
                print("❌ 게시물 목록 테이블 (tbody) 로딩 실패. 페이지에 게시물이 없거나 구조 변경됨.")
                break # 게시물 목록을 찾지 못하면 반복 중단

            article_rows = tbodys[-1].find_elements(By.TAG_NAME, "tr")
            if not article_rows:
                print("❌ 현재 페이지에 게시물이 없습니다.")
                break # 게시물이 없으면 반복 중단

        except TimeoutException as e:
            print(f"❌ 게시물 목록 로딩 실패 (Timeout): {e}")
            break
        except Exception as e:
            print(f"❌ 게시물 목록 로딩 중 알 수 없는 오류 발생: {e}")
            break


        processed_count_on_page = 0 # 현재 페이지에서 처리한 게시물 수
        for i in range(len(article_rows)):
            print(f"  └▷ {i+1}번째 게시물 처리 중...                      ", end="\r")

            # 게시물 상세 보기 진입 전, 다시 한번 현재 URL이 정상적인지 확인
            if "google.com" in driver.current_url:
                print("🚨 게시물 처리 중 튕김 감지 → 페이지 복구 시도")
                driver.get(target_url) # 원래 목록 페이지로 복귀
                time.sleep(3)
                handle_login(driver, wait)
                # 중요한 부분: 페이지 복구 후에는 article_rows가 stale해질 수 있으므로 다시 찾아야 합니다.
                try:
                    tbodys = safe_wait(driver, EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table.article-table tbody")))
                    article_rows = tbodys[-1].find_elements(By.TAG_NAME, "tr")
                except TimeoutException:
                    print("❌ 복구 후 게시물 목록 재로딩 실패. 다음 페이지로 넘어갑니다.")
                    break # 현재 페이지 처리를 중단하고 다음 페이지로 넘어감 (또는 전체 중단)

            try:
                # 중요: 각 게시물을 클릭하기 전에 현재 페이지의 article_rows를 다시 가져와 stale 에러 방지
                # driver.back() 후 다시 이 루프로 돌아올 때 요소가 유효한지 확인.
                # 그러나 이 부분은 for 루프 밖에서 처리하는 것이 더 효율적일 수 있습니다.
                # 현재는 driver.back() 후 재진입 시 다시 tbodys와 article_rows를 가져오므로 괜찮습니다.
                row = article_rows[i] # 이전에 찾은 요소를 사용

                date_element = row.find_element(By.CSS_SELECTOR, "td.td_normal.type_date")
                date_str = date_element.text.strip()
                collected_date = datetime.now().date()
                article_date = parse_cafe_date(date_str, collected_date)

                if article_date < one_year_ago:
                    print(f"✅ {article_date.strftime('%Y.%m.%d')} 게시물이 {one_year_ago.strftime('%Y.%m.%d')} 이전이므로 수집 종료합니다.")
                    stop_collecting = True
                    break # for 루프 탈출 (현재 페이지 처리 중단)

                link_element = row.find_element(By.CSS_SELECTOR, "a.article")
                article_url = link_element.get_attribute("href") # URL 미리 저장
                link_element.click() # 게시물 링크 클릭

                # 게시물 상세 페이지로 이동 후 iframe 전환
                safe_wait(driver, EC.frame_to_be_available_and_switch_to_it((By.ID, "cafe_main")))
                print("    └ iframe으로 전환 완료")

                # 게시물 본문 스크롤
                try:
                    last_height = driver.execute_script("return document.body.scrollHeight")
                    while True:
                        driver.execute_script("window.scrollBy(0, 800);") # 좀 더 크게 스크롤
                        time.sleep(1) # 스크롤 후 내용 로딩 대기
                        new_height = driver.execute_script("return document.body.scrollHeight")
                        if new_height == last_height: # 더 이상 스크롤할 내용이 없으면 종료
                            break
                        last_height = new_height
                    print("    └ 게시물 본문 스크롤 완료")
                except Exception as e:
                    print(f"❌ 반복 스크롤 실패: {e}")

                # 제목과 본문 추출
                title = safe_wait(driver, EC.presence_of_element_located((By.CSS_SELECTOR, ".title_text"))).text.strip()
                # .se-main-container (새 에디터) 또는 .content.CafeViewer (구 에디터)
                content = safe_wait(driver, EC.presence_of_element_located((By.CSS_SELECTOR, ".se-main-container, .content.CafeViewer"))).text.strip()

                # 댓글 추출
                reply_list = []
                try:
                    # 댓글 목록이 모두 로드될 때까지 기다리기
                    safe_wait(driver, EC.presence_of_element_located((By.CSS_SELECTOR, "div.CommentBox")), timeout=5) # 댓글 박스 대기
                    replies = driver.find_elements(By.CSS_SELECTOR, "div.CommentBox > div:nth-child(2) > ul > li")
                    for reply in replies:
                        try:
                            # 대댓글 등을 포함할 수 있으므로, .comment_text_box 내의 모든 텍스트를 가져오도록 수정
                            reply_text_elements = reply.find_elements(By.CSS_SELECTOR, "div.comment_text_box p span, div.comment_text_box p")
                            reply_text = ' '.join([el.text for el in reply_text_elements if el.text]).strip()
                            if reply_text: # 빈 댓글 제외
                                reply_list.append(reply_text)
                        except:
                            reply_list.append("(삭제된 댓글 또는 비정상 구조의 댓글)")
                except TimeoutException:
                    print("    └ 댓글 로딩 지연 (댓글 없음으로 처리)", end="\r")
                except Exception as e:
                    print(f"    └ 댓글 추출 중 오류 발생: {e}", end="\r")

                # CSV 파일에 저장
                writer.writerow([
                    article_date.strftime('%Y.%m.%d'),
                    ' '.join(title.split()), # 여러 공백 제거
                    ' '.join(content.split()), # 여러 공백 제거
                    ' | '.join(reply_list),
                    article_url # 미리 저장한 URL 사용
                ])
                processed_count_on_page += 1
                print(f"  {processed_count_on_page}번째 게시물 ✅ 저장 완료: {title[:30]}...", end="\r")
                time.sleep(1) # 너무 빠르게 다음 게시물로 넘어가지 않도록 잠시 대기

            except TimeoutException as e:
                print(f"    ❌ 게시물 상세 정보 로딩 실패 (Timeout): {e}")
            except NoSuchElementException as e:
                print(f"    ❌ 게시물 상세 정보 중 요소 찾기 실패: {e}")
            except Exception as e:
                print(f"    ❌ 게시물 처리 중 알 수 없는 오류 발생: {e}")
            finally:
                try:
                    driver.switch_to.default_content() # 다시 메인 프레임으로 전환
                    driver.back() # 게시물 목록 페이지로 돌아가기
                    time.sleep(3) # 페이지 로드 대기
                except Exception as e:
                    print(f"⚠️ 뒤로 가기 또는 메인 프레임 전환 실패: {e}")
                    # 만약 driver.back()이 실패하면, 다음 루프에서 target_url로 직접 이동하게 될 것입니다.
                    # 하지만 이 시점에서 더 이상 진행할 수 없다고 판단되면 break를 고려할 수도 있습니다.
                    pass # 계속 진행 시도

        if stop_collecting: # 게시물 날짜 조건에 따라 수집 중단 플래그가 설정되면
            break # while 루프 탈출 (전체 크롤링 종료)

        # --- 페이지네이션 처리 ---
        try:
            # 현재 보이는 페이지 번호 버튼들 수집
            page_numbers_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".Pagination .btn.number")))
            
            next_page_button_to_click = None

            # 현재 페이지 번호 (활성화된 버튼) 찾기
            current_page_num = -1
            for btn in page_numbers_elements:
                if btn.get_attribute("aria-pressed") == "true":
                    current_page_num = int(btn.text)
                    break
            
            # 현재 페이지 세트 내에서 다음 페이지 번호가 있으면 클릭
            found_next_in_set = False
            for btn in page_numbers_elements:
                try:
                    btn_num = int(btn.text)
                    if btn_num == current_page_num + 1:
                        next_page_button_to_click = btn
                        found_next_in_set = True
                        break
                except ValueError:
                    # '...' 등의 텍스트가 있는 버튼은 int 변환 실패하므로 건너뜀
                    continue
            
            # 현재 페이지 세트의 마지막 페이지(예: 10페이지)이고 '다음 페이지 세트' 버튼(>)이 있다면 그것을 클릭
            if not found_next_in_set:
                try:
                    # 다음 페이지 세트 버튼 (꺽쇠 모양)
                    next_set_button = driver.find_element(By.CSS_SELECTOR, ".Pagination .btn.type_next")
                    
                    # '다음' 버튼이 비활성화되어 있는지 확인
                    if "disabled" in next_set_button.get_attribute("class"):
                        print("마지막 페이지 세트입니다. 더 이상 페이지가 없습니다.")
                        break # 더 이상 이동할 페이지가 없으므로 루프 종료
                    
                    next_page_button_to_click = next_set_button
                    print("다음 페이지 세트로 이동합니다.")
                    
                except Exception:
                    # 다음 세트 버튼이 아예 없는 경우 (마지막 페이지이거나 10페이지 이하)
                    print("더 이상 페이지를 찾을 수 없습니다. (다음 세트 버튼 없음)")
                    break

            if next_page_button_to_click:
                wait.until(EC.element_to_be_clickable(next_page_button_to_click)).click()
                print(f"페이지 이동: {next_page_button_to_click.text if next_page_button_to_click.text else '다음 세트'}")
                time.sleep(3) # 페이지 로딩 대기 시간을 충분히 줍니다.
            else:
                print("더 이상 이동할 페이지가 없어 종료합니다.")
                break

        except Exception as e:
            print(f"페이지네이션 처리 중 오류 발생 또는 마지막 페이지 도달: {e}")
            break # 페이지네이션 처리 중 문제가 발생하면 루프 종료

print(f"\n✅ 모든 게시물 수집 완료 → '{csv_filename}'에 저장됨")
driver.quit()


네이버카페 게시판 크롤링 시작
🚨 비정상 URL 감지 또는 초기 진입 → 대상 URL로 이동
✅ 로그인 버튼 없음 (로그인 상태로 추정): <function element_to_be_clickable.<locals>._predicate at 0x000001DB5D111310>
📄 페이지 1 진입 시도: https://cafe.naver.com/f-e/cafes/19997929/menus/809?page=1&size=50
    └ iframe으로 전환 완료                      t_to_be_clickable.<locals>._predicate at 0x000001DB5D111310>
    └ 게시물 본문 스크롤 완료
    ❌ 게시물 처리 중 알 수 없는 오류 발생: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=137.0.7151.104); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x0xca3b03+62899]
	GetHandleVerifier [0x0xca3b44+62964]
	(No symbol) [0x0xad10f3]
	(No symbol) [0x0xad78e0]
	(No symbol) [0x0xad9c0a]
	(No symbol) [0x0xad9c87]
	(No symbol) [0x0xb190a4]
	(No symbol) [0x0xb19bab]
	(No symbol) [0x0xb0ef51]
	(No symbol) [0x0xb3e554]
	(No symbol) [0x0xb0ee74]
	(No symbol) [0x

    ❌ 게시물 처리 중 알 수 없는 오류 발생: Message: stale element reference: stale element not found
  (Session info: chrome=137.0.7151.104); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x0xca3b03+62899]
	GetHandleVerifier [0x0xca3b44+62964]
	(No symbol) [0x0xad10f3]
	(No symbol) [0x0xae2bc1]
	(No symbol) [0x0xae1ca0]
	(No symbol) [0x0xad8392]
	(No symbol) [0x0xad6891]
	(No symbol) [0x0xad9c0a]
	(No symbol) [0x0xad9c87]
	(No symbol) [0x0xb190a4]
	(No symbol) [0x0xb19bab]
	(No symbol) [0x0xb0ef51]
	(No symbol) [0x0xb3e554]
	(No symbol) [0x0xb0ee74]
	(No symbol) [0x0xb3e784]
	(No symbol) [0x0xb5fd81]
	(No symbol) [0x0xb3e306]
	(No symbol) [0x0xb0d670]
	(No symbol) [0x0xb0e4e4]
	GetHandleVerifier [0x0xf04793+2556483]
	GetHandleVerifier [0x0xeffd02+2537394]
	GetHandleVerifier [0x0xcca2fa+220586]
	GetHandleVerifier [0x0xcbaae8+157080]
	GetHandleVerifier [0x0xcc141d+

    ❌ 게시물 처리 중 알 수 없는 오류 발생: Message: stale element reference: stale element not found
  (Session info: chrome=137.0.7151.104); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x0xca3b03+62899]
	GetHandleVerifier [0x0xca3b44+62964]
	(No symbol) [0x0xad10f3]
	(No symbol) [0x0xae2bc1]
	(No symbol) [0x0xae1ca0]
	(No symbol) [0x0xad8392]
	(No symbol) [0x0xad6891]
	(No symbol) [0x0xad9c0a]
	(No symbol) [0x0xad9c87]
	(No symbol) [0x0xb190a4]
	(No symbol) [0x0xb19bab]
	(No symbol) [0x0xb0ef51]
	(No symbol) [0x0xb3e554]
	(No symbol) [0x0xb0ee74]
	(No symbol) [0x0xb3e784]
	(No symbol) [0x0xb5fd81]
	(No symbol) [0x0xb3e306]
	(No symbol) [0x0xb0d670]
	(No symbol) [0x0xb0e4e4]
	GetHandleVerifier [0x0xf04793+2556483]
	GetHandleVerifier [0x0xeffd02+2537394]
	GetHandleVerifier [0x0xcca2fa+220586]
	GetHandleVerifier [0x0xcbaae8+157080]
	GetHandleVerifier [0x0xcc141d+

    ❌ 게시물 처리 중 알 수 없는 오류 발생: Message: stale element reference: stale element not found
  (Session info: chrome=137.0.7151.104); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x0xca3b03+62899]
	GetHandleVerifier [0x0xca3b44+62964]
	(No symbol) [0x0xad10f3]
	(No symbol) [0x0xae2bc1]
	(No symbol) [0x0xae1ca0]
	(No symbol) [0x0xad8392]
	(No symbol) [0x0xad6891]
	(No symbol) [0x0xad9c0a]
	(No symbol) [0x0xad9c87]
	(No symbol) [0x0xb190a4]
	(No symbol) [0x0xb19bab]
	(No symbol) [0x0xb0ef51]
	(No symbol) [0x0xb3e554]
	(No symbol) [0x0xb0ee74]
	(No symbol) [0x0xb3e784]
	(No symbol) [0x0xb5fd81]
	(No symbol) [0x0xb3e306]
	(No symbol) [0x0xb0d670]
	(No symbol) [0x0xb0e4e4]
	GetHandleVerifier [0x0xf04793+2556483]
	GetHandleVerifier [0x0xeffd02+2537394]
	GetHandleVerifier [0x0xcca2fa+220586]
	GetHandleVerifier [0x0xcbaae8+157080]
	GetHandleVerifier [0x0xcc141d+

    ❌ 게시물 처리 중 알 수 없는 오류 발생: Message: stale element reference: stale element not found
  (Session info: chrome=137.0.7151.104); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x0xca3b03+62899]
	GetHandleVerifier [0x0xca3b44+62964]
	(No symbol) [0x0xad10f3]
	(No symbol) [0x0xae2bc1]
	(No symbol) [0x0xae1ca0]
	(No symbol) [0x0xad8392]
	(No symbol) [0x0xad6891]
	(No symbol) [0x0xad9c0a]
	(No symbol) [0x0xad9c87]
	(No symbol) [0x0xb190a4]
	(No symbol) [0x0xb19bab]
	(No symbol) [0x0xb0ef51]
	(No symbol) [0x0xb3e554]
	(No symbol) [0x0xb0ee74]
	(No symbol) [0x0xb3e784]
	(No symbol) [0x0xb5fd81]
	(No symbol) [0x0xb3e306]
	(No symbol) [0x0xb0d670]
	(No symbol) [0x0xb0e4e4]
	GetHandleVerifier [0x0xf04793+2556483]
	GetHandleVerifier [0x0xeffd02+2537394]
	GetHandleVerifier [0x0xcca2fa+220586]
	GetHandleVerifier [0x0xcbaae8+157080]
	GetHandleVerifier [0x0xcc141d+

    ❌ 게시물 처리 중 알 수 없는 오류 발생: Message: stale element reference: stale element not found
  (Session info: chrome=137.0.7151.104); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x0xca3b03+62899]
	GetHandleVerifier [0x0xca3b44+62964]
	(No symbol) [0x0xad10f3]
	(No symbol) [0x0xae2bc1]
	(No symbol) [0x0xae1ca0]
	(No symbol) [0x0xad8392]
	(No symbol) [0x0xad6891]
	(No symbol) [0x0xad9c0a]
	(No symbol) [0x0xad9c87]
	(No symbol) [0x0xb190a4]
	(No symbol) [0x0xb19bab]
	(No symbol) [0x0xb0ef51]
	(No symbol) [0x0xb3e554]
	(No symbol) [0x0xb0ee74]
	(No symbol) [0x0xb3e784]
	(No symbol) [0x0xb5fd81]
	(No symbol) [0x0xb3e306]
	(No symbol) [0x0xb0d670]
	(No symbol) [0x0xb0e4e4]
	GetHandleVerifier [0x0xf04793+2556483]
	GetHandleVerifier [0x0xeffd02+2537394]
	GetHandleVerifier [0x0xcca2fa+220586]
	GetHandleVerifier [0x0xcbaae8+157080]
	GetHandleVerifier [0x0xcc141d+

    ❌ 게시물 처리 중 알 수 없는 오류 발생: Message: stale element reference: stale element not found
  (Session info: chrome=137.0.7151.104); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x0xca3b03+62899]
	GetHandleVerifier [0x0xca3b44+62964]
	(No symbol) [0x0xad10f3]
	(No symbol) [0x0xae2bc1]
	(No symbol) [0x0xae1ca0]
	(No symbol) [0x0xad8392]
	(No symbol) [0x0xad6891]
	(No symbol) [0x0xad9c0a]
	(No symbol) [0x0xad9c87]
	(No symbol) [0x0xb190a4]
	(No symbol) [0x0xb19bab]
	(No symbol) [0x0xb0ef51]
	(No symbol) [0x0xb3e554]
	(No symbol) [0x0xb0ee74]
	(No symbol) [0x0xb3e784]
	(No symbol) [0x0xb5fd81]
	(No symbol) [0x0xb3e306]
	(No symbol) [0x0xb0d670]
	(No symbol) [0x0xb0e4e4]
	GetHandleVerifier [0x0xf04793+2556483]
	GetHandleVerifier [0x0xeffd02+2537394]
	GetHandleVerifier [0x0xcca2fa+220586]
	GetHandleVerifier [0x0xcbaae8+157080]
	GetHandleVerifier [0x0xcc141d+

페이지네이션 처리 중 오류 발생 또는 마지막 페이지 도달: Message: 


✅ 모든 게시물 수집 완료 → 'test'에 저장됨
